# Building Snort

Snort is an IDS/IPS solution provided by Cisco Talos.

For this build, I am building on
- VMware (ESXi 7.0u3)
- Docker on Ubuntu (Desktop 22.04.4)
- Installing SnortV3
- I will test with Wireshark and Metaspoit to confirm operation

<hr>

## Building the Ubuntu Box

For this lab, I'm building a Docker Box on Ubuntu. Technically you can run multiple solutions on Docker, but I'm still old when it comes to separation of services. Besides, we'll be doing things to the network & Docker that are really bad practice.

I'm building a VM with 2xCPU, 4GB RAM and 32GB Storage. As always I have thin provisioned my VMs as I normally overprovision for the VM requirements. A key section here is that I've allocated the NIC to my INTERNAL network. This INTERNAL is internal to the virtual enviroment and isolated from my EXTERNAL home network via the pfSense router. Don't want those two things mixing...

![Snort_VMspecs.png](../images/HomeLab/Snort_VMspecs.png)

For the OS install, just follow the defaults (I'll go through from a minimal installation). Remember to store your passwords in a password manager :-). While you're waiting for the wheels to spin, we need to make a change to the virtual network.

### Promiscous Switch

While I'm sure you know this, for packet sniffing to run, we need to recieve the packets. Who would have thought. The problem with this is that virtual switches, well, switch traffic between specific ports. Data between 2 VMs on the same virtual switch is sent between just those two VMs; our sniffer gets left out of the conversation. To bypass this, we need to tell the vSwitch to send data to everyone on the vSwitch (functionally turning the switch into a hub). Most NICs will just drop the traffic they recieve as it's not for their MAC, however our sniffer will also be configured to pick this up. If this sounds excessive, you are correct, in a large vSwitch you will end up with data everywhere, bad practice. 
Instead we can assign this mode to the Port Group; Internal as you saw earlier.<br>

![Snort_PortGroup.png](../images/HomeLab/Snort_PortGroup.png)<br>

You can then set Promiscous mode under security.<br>
![Snort_PortGroupPromiscous.png](../images/HomeLab/Snort_PortGroupPromiscous.png)<br>

Now all we need to do is wait for the VM to finish building.....

<hr>

## Building Docker

Once the VM has built and logged in, we need to run some standard Ubuntu things
- If there are any updates, run them (you'll be prompted)
- Let Firefox install

We also need to ensure virtualisation is available to docker. Basically allowing the VM to host VMs itself. This technology is known by different names depending on your host CPU (Intel V-T or AMD ....). Fortunately for us, it's the same thing to VMware. Shut down your VM then edit it's settings.<br>

Check the 'Hardware Virtualisation' checkbox then save and start your VM back up.

![Snort_CPUflag.png](../images/HomeLab/Snort_CPUflag.png)<br>

### Install
Docker has plenty of documentation available for installation.